In [8]:
%load_ext autoreload
%autoreload 2

from src.settings import Paths, Settings
from src.data_loader import CLEARDataLoader
from ieeg_data_loader.data import iEEGDataLoader
import pandas as pd



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
settings = Settings()  # Initialize settings object
settings.load_settings()  # Load settings from a JSON file
settings.dataset = 'clear'
settings.patient = [f'p{i+1:02}' for i in range(18)]
print(f"Patients: {settings.patient}")

Patient: ['p09']
Patients: ['p01', 'p02', 'p03', 'p04', 'p05', 'p06', 'p07', 'p08', 'p09', 'p10', 'p11', 'p12', 'p13', 'p14', 'p15', 'p16', 'p17', 'p18']


In [13]:
paths = Paths(settings)  # Initialize paths object with loaded settings
paths.load_device_paths()  # Load device-specific paths
paths.create_paths()  # Create any necessary file paths

In [14]:
features_raw_df_list = []
meta_data_all = []
for patient in settings.patient:
    # Load EEG dataset using configured settings and paths
    dataset = CLEARDataLoader(paths=paths, settings=settings)
    dataset.load_data(patient_ids=patient)  # Load EEG data for specified patients
    data_loader = iEEGDataLoader(patient=patient,
                                     target_class='color',
                                     prepared_dataset_path=paths.raw_dataset_path,
                                     task=settings.dataset_task,
                                     file_format='npy')
    dataset_list = data_loader.load_data()
    
    for dataset in dataset_list:
        meta_data = {
            'Patient': patient,
            'Block': dataset.meta[-1],
            'Task': settings.dataset_task,
            'Num_Channels': len(dataset.channel_name),
            'Num_Trials': dataset.data.shape[0]
        }
        meta_data_all.append(meta_data)
        


0: Data p01_task2_block2_CLEAR.pkl 	 Block Number: [2.] 	Number of trials: 100
1: Data p01_task2_block3_CLEAR.pkl 	 Block Number: [3.] 	Number of trials: 31
2: Data p01_task2_block4_CLEAR.pkl 	 Block Number: [4.] 	Number of trials: 100
3: Data p01_task2_block5_CLEAR.pkl 	 Block Number: [5.] 	Number of trials: 99
trials with trial_end annotation problem [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95
 96 97 98]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0.] is replaced with 0.8
task m_sequence : 

In [16]:
df_info = pd.DataFrame(meta_data_all)
df_info.to_csv(paths.eda_results + 'm_sequence_task_summary.csv')